In [1]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.image import ContainerImage
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig,Model
import sys

In [2]:
ws = Workspace.from_config()

In [3]:
webservice_name = 'turbofan-rul'

run = [x for x in ws.experiments['gbr-turbofan'].get_runs()][0]
run_metrics = run.get_metrics()

print(run)

model = run.register_model(model_name = "turbofan-rul", model_path = "outputs/model.pkl", tags={"mae":run_metrics["mae"],"python version": sys.version[0:6]})

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1,auth_enabled=True,enable_app_insights=True)

Run(Experiment: gbr-turbofan,
Id: 250790bf-8f89-47db-9fa8-b193b6d3b95b,
Type: None,
Status: Completed)


In [4]:
myenv = Environment.get(ws,name="sklearn_20_2")
myenv.inferencing_stack_version = "latest"

#myenv = Environment(name="turbofan")
#conda_dep = CondaDependencies("turbofan.yml")

#myenv.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script="score.py",
                                   environment=myenv)
                               
try:
    service = Webservice(ws, webservice_name)        
    service.delete()    
except Exception as e:
    print("No Existing Service")

service = Model.deploy(ws, webservice_name, [model], inference_config, deployment_config)

#service.get_logs()
service.wait_for_deployment(show_output = True)

#print(service.get_keys())
print(service.scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-09-27 14:02:01+00:00 Creating Container Registry if not exists.
2021-09-27 14:02:01+00:00 Registering the environment.
2021-09-27 14:02:02+00:00 Use the existing image.
2021-09-27 14:02:03+00:00 Generating deployment configuration.
2021-09-27 14:02:03+00:00 Submitting deployment to compute.
2021-09-27 14:02:09+00:00 Checking the status of deployment turbofan-rul..
2021-09-27 14:12:28+00:00 Checking the status of inference endpoint turbofan-rul.
Succeeded
ACI service creation operation finished, operation "Succeeded"
http://e0c092f2-5f06-4c18-8730-decb67ce980e.southcentralus.azurecontainer.io/score
